In [1]:
import tensorflow as tf
from tensorflow.python.layers.core import Dense
import numpy as np
import os


print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.0.0


In [2]:
checkpoint1 = "question_model.ckpt" 
checkpoint2 = "answer_model.ckpt" 
checkpoint3 = "force_answer_model.ckpt" 
tensorboard_path = "/Users/unoboros"

In [3]:
question_netowrk = tf.Graph()
with question_netowrk.as_default():
    
    board_1 = tf.placeholder(tf.float32, [1,1+10+10+11], name='board_1')
    target_1 = tf.placeholder(tf.float32, [1,11+1], name='target_1')
    nn1 = tf.layers.dense(board_1, 100, activation=tf.nn.sigmoid, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    nn2 = tf.layers.dense(nn1, 100, activation=tf.nn.sigmoid, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    nn3 = tf.layers.dense(nn2, 100+1, activation=tf.nn.sigmoid, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    
    question_prior = tf.nn.softmax(nn3[:,:11])
    question_v = nn3[:,11]
    prior = question_prior
    v = question_v
    pi = target_1[:,:11]
    z = target_1[:,11]
    
    question_cost = tf.reduce_mean((z - v)**2 + tf.nn.softmax_cross_entropy_with_logits(logits=prior, labels=pi))
    question_optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(question_cost)
    
    init_1 = tf.global_variables_initializer()
    
question_sess = tf.Session(graph=question_netowrk)
question_sess.run(init_1)

In [4]:
answer_netowrk = tf.Graph()
with answer_netowrk.as_default():
    board_2 = tf.placeholder(tf.float32, [None,10+10+11+11], name='board_2')
    target_2= tf.placeholder(tf.float32, [None,3+1], name='target_2')
    nn4 = tf.layers.dense(board_2, 100, activation=tf.nn.sigmoid,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    nn5 = tf.layers.dense(nn4, 50, activation=tf.nn.sigmoid,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    nn6 = tf.layers.dense(nn5, 3+1, activation=tf.nn.sigmoid,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    answer_prior = tf.nn.softmax(nn6[:,:3])
    answer_v = nn6[:,3]
    prior = answer_prior
    v = answer_v
    pi = target_2[:,:3]
    z = target_2[:,3]
    
    answer_cost = tf.reduce_mean((z - v)**2 + tf.nn.softmax_cross_entropy_with_logits(logits=prior, labels=pi))
    answer_optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(answer_cost)
    init_2 = tf.global_variables_initializer()
answer_sess = tf.Session(graph=answer_netowrk)
answer_sess.run(init_2)

In [5]:
force_answer_netowrk = tf.Graph()
with force_answer_netowrk.as_default():
    board_3 = tf.placeholder(tf.float32, [None,10+10+11+11], name='board_3')
    target_3 = tf.placeholder(tf.float32, [None,2+1], name='target_3')
    nn7 = tf.layers.dense(board_3, 100, activation=tf.nn.sigmoid,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    nn8 = tf.layers.dense(nn7, 50, activation=tf.nn.sigmoid,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    nn9 = tf.layers.dense(nn8, 2+1, activation=tf.nn.sigmoid,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    force_answer_prior = tf.nn.softmax(nn9[:,:2])
    force_answer_v = nn9[:,2]
    prior = force_answer_prior
    v = force_answer_v
    pi = target_3[:,:2]
    z = target_3[:,2]
    
    force_answer_cost = tf.reduce_mean((z - v)**2 + tf.nn.softmax_cross_entropy_with_logits(logits=prior, labels=pi))
    force_answer_optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(force_answer_cost)
    init_3 = tf.global_variables_initializer()
force_answer_sess = tf.Session(graph=force_answer_netowrk)
force_answer_sess.run(init_3)

In [6]:
class HSRState:
    """ A state of the game, i.e. the game board.
        (k,q,n)
        k : number of jars left
        q : number of questions left
        n : number of total rungs 
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [2,3,5,-1] # k, q, n_max, n chosen as question
        self.reversedRole = False
        self.P2firstMove = True
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = HSRState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        st.reversedRole = self.reversedRole
        st.P2firstMove = self.P2firstMove
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        k = self.board[0]
        q = self.board[1]
        n_max = self.board[2]
        n = self.board[3]
        assert n <= n_max
        if not self.reversedRole:
            if self.playerJustMoved == 2:
                self.board[3] = move
            if self.playerJustMoved == 1:
                self.P2firstMove = False
                if move[0]=='accept':
                    if move[1] == 'yes':
                        # break
                        k -= 1
                        q -= 1
                        n_max = n-1
                        n = -1
                        self.board = [k,q,n_max,n]

                    elif move[1] == 'no':
                        #not break
                        q -= 1
                        n_max = n_max-n
                        n = -1
                        self.board = [k,q,n_max,n]

                elif move[0]=='reject':
                    self.reversedRole = True
                    if n < n_max:
                        n += 1
                    self.board[3] = n
        else:
            if self.playerJustMoved == 2:
                if move[1] == 'yes':
                    # break
                    k -= 1
                    q -= 1
                    n_max = n-1
                    n = -1
                    self.board = [k,q,n_max,n]

                elif move[1] == 'no':
                    #not break
                    q -= 1
                    n_max = n_max-n
                    n = -1
                    self.board = [k,q,n_max,n]
            if self.playerJustMoved == 1:
                self.P2firstMove = False
                self.board[3] = move

        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        k = self.board[0]
        q = self.board[1]
        n_max = self.board[2]
        n = self.board[3]
        if not self.reversedRole:
            if n_max == 0:
                return []
            if k==0 or q==0:
                return []
            if self.playerJustMoved == 2:
                return range(1,n_max+1)
            if self.playerJustMoved == 1:
                if n < n_max and self.P2firstMove:
                    return [('accept','yes'),('accept','no'),('reject','')]
                else:
                    return [('accept','yes'),('accept','no')]
        else:
            if n_max == 0:
                return []
            if k==0 or q==0:
                return []
            if self.playerJustMoved == 1:
                return range(1,n_max+1)
            if self.playerJustMoved == 2:
                return [('accept','yes'),('accept','no')]

    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        self.P2firstMove = True
        n_max = self.board[2]
        if not self.reversedRole:
            if playerjm == 1:
                return 1 if n_max == 0 else 0
            if playerjm == 2:
                return 0 if n_max == 0 else 1
        else:
            if playerjm == 1:
                return 0 if n_max == 0 else 1
            if playerjm == 2:
                return 1 if n_max == 0 else 0


    def __repr__(self):
        s = ''
        for x in self.board:
            s += str(x) +','

        return s[:-1]

In [7]:
def neural_evaluator(state):
    """
    if role has not been reversed, then player 1 always use question neural network and 2 always use answer neural network.
    if role has been reversed, then player 1 will use forced answer neural network, 2 will use question neural network.
    """
    if not state.reversedRole:
        if state.playerJustMoved == 2:
            # player 1 play
            f = 1 if state.P2firstMove else 0
            k,q,n_max,_ = state.board
            prior, v = question_sess.run([question_prior,question_v],{board_1: [getBoardOneHot(f,k,q,n_max)]})
            return prior , v
        else:
            # player 2 play
            k,q,n_max,n = state.board
            prior, v = answer_sess.run([answer_prior,answer_v],{board_2: [getBoardOneHot2(k,q,n_max,n)]})
            return prior , v
    else:
        if state.playerJustMoved == 2:
            # player 1 play
            k,q,n_max,n = state.board
            prior, v = force_answer_sess.run([force_answer_prior, force_answer_v],{board_3: [getBoardOneHot2(k,q,n_max,n)]})
            return prior , v
        else:
            # player 2 play
            f = 1 if state.P2firstMove else 0
            k,q,n_max,_ = state.board
            prior, v = question_sess.run([question_prior, question_v],{board_1: [getBoardOneHot(f,k,q,n_max)]})
            return prior , v
        

In [8]:
def getBoardOneHot(f,k,q,n_max):
    # [None,1+10+10+100]
    f0 = np.zeros(shape=(1), dtype=np.float32)
    k0 = np.zeros(shape=(10), dtype=np.float32)
    q0 = np.zeros(shape=(10), dtype=np.float32)
    n_max0 = np.zeros(shape=(11), dtype=np.float32)
    f0[0] = f
    k0[k-1] = 1
    q0[q-1] = 1
    n_max0[n_max-1]=1
    return np.concatenate((f0, k0, q0, n_max0), axis=0)
def getBoardOneHot2(k,q,n_max,n):
    # [None,10+10+100+100]
    k0 = np.zeros(shape=(10), dtype=np.float32)
    q0 = np.zeros(shape=(10), dtype=np.float32)
    n_max0 = np.zeros(shape=(11), dtype=np.float32)
    n0 = np.zeros(shape=(11), dtype=np.float32)
    k0[k-1] = 1
    q0[q-1] = 1
    n_max0[n_max-1]=1
    n0[n-1] = 1
    return np.concatenate((k0, q0, n_max0, n0), axis=0)
    

In [9]:
import random
def moveSampler(notes, pi):
    x = random.random()
    acc = 0
    for index in range(len(pi)):
        if x <= acc + pi[index]:
            return notes[index].move
        acc += pi[index]

In [21]:
from math import *
import random

class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement PUCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass

class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.prior = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def PUCTSelectChild(self):
        """ Use the PUCK formula to select a child node. Often a constant PUCTK is applied so we have
            lambda c: Q + U to vary the amount of exploration versus exploitation.

            Q = c.wins/c.visits
            U = PUCTK * prior * sqrt(self.visits)/c.visits

            (according to the AplhaZero paper)

        """
        PUCTK = 1
        x = zip(sorted(self.childNodes, key = lambda c:c.move), self.prior[:len(self.childNodes)][0])
        s = sorted(x, key = lambda c: c[0].wins*1.0/c[0].visits+ PUCTK * c[1] * sqrt(self.visits)/(1+c[0].visits) if c[0].visits!=0 else PUCTK * c[1] * sqrt(self.visits)/(1+c[0].visits))[-1]
        return s[0]
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result #0 if result < 0.5 else 1

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s
    
    def PriorToString(self):
        s = ""
        for c in self.prior:
             s += str(c) + "\n"
        return s


def PUCT(rootstate, itermax, verbose = False):
    """ Conduct a PUCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
#         print('iter:{}'.format(i))
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.PUCTSelectChild()
            state.DoMove(node.move)

        # Expand and Evaluate
        prior, v = neural_evaluator(state)
        node.prior = prior
        
#         print('v={}'.format(v))
#         print('prior={}'.format(prior))
        
        while node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves)
            state_next = state.Clone()
            state_next.DoMove(m)
            c_node = node.AddChild(m,state_next) # add child and descend tree
            # Backpropagate
            while c_node != None: # backpropagate from the expanded node and work back to the root node
                c_node.Update(v) # state is terminal. Update node with result from POV of node.playerJustMoved
                c_node = c_node.parentNode
        

    # Output some information about the tree - can be omitted
    # if (verbose): print rootnode.TreeToString(0)
    # else: print rootnode.ChildrenToString()
    
#     print rootnode.TreeToString(0)
    
    # Here we should return distribution of the moves:
    # TODO: need use sampling instead of maximum.
    root_visit = sum(map(lambda c:c.visits,rootnode.childNodes))
    tau = 1
    pi = map(lambda c:(c.visits**tau)*1.0/(rootnode.visits**tau), sorted(rootnode.childNodes, key = lambda c:c.move))
    
    move = moveSampler(sorted(rootnode.childNodes, key = lambda c:c.move),pi)
    
#     print pi
    
#     print sorted(rootnode.childNodes, key = lambda c: c.visits)

#     return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move, pi
    return move,pi

def PUCTPlayGame():
    """ Play a sample game between two PUCT players where each player gets a different number 
        of PUCT iterations (= simulations = tree nodes).
    """
    winner = -1
    # state = OthelloState(8) # uncomment to play Othello on a square board of the given size
    # state = OXOState() # uncomment to play OXO
    # state = NimState(15) # uncomment to play Nim with the given number of starting chips
    # state = C4State()   #Uncomment to simulate a full game of connect 4
    state = HSRState()    #Uncomment to simulate a full game of HSR
    
    history_s = []
    history_pi=[]
    history_z=[]
    
    while (state.GetMoves() != []):
#         print(str(state))
        if state.playerJustMoved == 1:
            m,pi = PUCT(rootstate = state, itermax = 5000, verbose = False) # play with values for itermax and verbose = True
        else:
            m,pi = PUCT(rootstate = state, itermax = 5000, verbose = False)
#         print(pi)
#         print("Player "+str(3-state.playerJustMoved)+" Move: " + str(m))
        history_s.append(state.Clone())
        history_pi.append(pi)
        state.DoMove(m)
    if state.GetResult(state.playerJustMoved) == 1.0:
#         print("Player " + str(state.playerJustMoved) + " wins!")
        winner = state.playerJustMoved
    elif state.GetResult(state.playerJustMoved) == 0.0:
#         print("Player " + str(3 - state.playerJustMoved) + " wins!")
        winner = 3 - state.playerJustMoved
    else: print("Nobody wins!")
#     print('final state:{}'.format(state))
#     print
    assert len(history_s) == len(history_pi)
    for step in reversed(range(len(history_s))):
        if winner == state.playerJustMoved:
            if (len(history_s) - step) % 2 == 1:
                history_z.append(1)
            else:
                history_z.append(0)
        else:
            if (len(history_s) - step) % 2 == 1:
                history_z.append(0)
            else:
                history_z.append(1)
    assert len(history_s) == len(history_z)
    history_z.reverse() 
    return winner , history_s, history_pi ,history_z

# we can train the neural network only after getting the result
# according to the AlphaZero paper, we need (s,pi,z)

result = {1:0,2:0}
for simulation in range(1000):
    winner, history_s, history_pi ,history_z = PUCTPlayGame()
    result[winner] += 1
    print str(result)+"\n"
#     print history_z
    for t in range(len(history_s)):
        state = history_s[t]
        pi = history_pi[t]
        z = history_z[t]
#         print(pi)
#         print(z)
#         print(state.board)
        if not state.reversedRole:
            if state.playerJustMoved == 2:
                # player 1 play
                f = 1 if state.P2firstMove else 0
                k,q,n_max,_ = state.board
                pi = pi+[0]*(11-len(history_pi[t]))
                _, costt = question_sess.run([question_optimizer,question_cost],{board_1: [getBoardOneHot(f,k,q,n_max)], target_1: [pi+[z]]})
#                 print("question cost:"+str(costt))
            else:
                # player 2 play
                k,q,n_max,n = state.board
                pi = pi+[0]*(3-len(history_pi[t]))
                _, costt = answer_sess.run([answer_optimizer,answer_cost],{board_2: [getBoardOneHot2(k,q,n_max,n)], target_2: [pi+[z]]})
#                 print("answer cost:"+str(costt))
        else:
            if state.playerJustMoved == 2:
                # player 1 play
                k,q,n_max,n = state.board
                pi = pi+[0]*(2-len(history_pi[t]))
                _, costt = force_answer_sess.run([force_answer_optimizer,force_answer_cost],{board_3: [getBoardOneHot2(k,q,n_max,n)], target_3: [pi+[z]]})
#                 print("force answer cost:"+str(costt))
            else:
                # player 2 play
                f = 1 if state.P2firstMove else 0
                k,q,n_max,_ = state.board
                pi = pi+[0]*(11-len(history_pi[t]))
                _, costt = question_sess.run([question_optimizer,question_cost],{board_1: [getBoardOneHot(f,k,q,n_max)], target_1: [pi+[z]]})
#                 print("question cost:"+str(costt))
                
question_sess.close()
answer_sess.close()
force_answer_sess.close()

print result
            
    

{1: 0, 2: 1}

{1: 1, 2: 1}

{1: 2, 2: 1}

{1: 3, 2: 1}

{1: 4, 2: 1}

{1: 5, 2: 1}

{1: 6, 2: 1}

{1: 7, 2: 1}

{1: 7, 2: 2}

{1: 8, 2: 2}

{1: 9, 2: 2}

{1: 9, 2: 3}

{1: 9, 2: 4}

{1: 9, 2: 5}

{1: 10, 2: 5}

{1: 11, 2: 5}

{1: 12, 2: 5}

{1: 13, 2: 5}

{1: 14, 2: 5}

{1: 15, 2: 5}

{1: 15, 2: 6}

{1: 15, 2: 7}

{1: 15, 2: 8}

{1: 16, 2: 8}

{1: 16, 2: 9}

{1: 17, 2: 9}

{1: 18, 2: 9}

{1: 19, 2: 9}

{1: 20, 2: 9}

{1: 21, 2: 9}

{1: 21, 2: 10}

{1: 22, 2: 10}

{1: 23, 2: 10}

{1: 24, 2: 10}

{1: 25, 2: 10}

{1: 26, 2: 10}

{1: 27, 2: 10}

{1: 28, 2: 10}

{1: 29, 2: 10}

{1: 30, 2: 10}

{1: 30, 2: 11}

{1: 31, 2: 11}

{1: 32, 2: 11}

{1: 32, 2: 12}

{1: 33, 2: 12}

{1: 34, 2: 12}

{1: 35, 2: 12}

{1: 36, 2: 12}

{1: 37, 2: 12}

{1: 38, 2: 12}

{1: 39, 2: 12}

{1: 40, 2: 12}

{1: 41, 2: 12}

{1: 42, 2: 12}

{1: 43, 2: 12}

{1: 44, 2: 12}

{1: 45, 2: 12}

{1: 46, 2: 12}

{1: 46, 2: 13}

{1: 46, 2: 14}

{1: 47, 2: 14}

{1: 48, 2: 14}

{1: 49, 2: 14}

{1: 49, 2: 15}

{1: 49, 2: 16}

{1: 

{1: 341, 2: 144}

{1: 342, 2: 144}

{1: 343, 2: 144}

{1: 344, 2: 144}

{1: 344, 2: 145}

{1: 345, 2: 145}

{1: 345, 2: 146}

{1: 346, 2: 146}

{1: 347, 2: 146}

{1: 348, 2: 146}

{1: 348, 2: 147}

{1: 349, 2: 147}

{1: 350, 2: 147}

{1: 351, 2: 147}

{1: 351, 2: 148}

{1: 352, 2: 148}

{1: 352, 2: 149}

{1: 353, 2: 149}

{1: 353, 2: 150}

{1: 353, 2: 151}

{1: 353, 2: 152}

{1: 354, 2: 152}

{1: 355, 2: 152}

{1: 356, 2: 152}

{1: 357, 2: 152}

{1: 358, 2: 152}

{1: 358, 2: 153}

{1: 358, 2: 154}

{1: 359, 2: 154}

{1: 359, 2: 155}

{1: 360, 2: 155}

{1: 360, 2: 156}

{1: 361, 2: 156}

{1: 362, 2: 156}

{1: 363, 2: 156}

{1: 363, 2: 157}

{1: 363, 2: 158}

{1: 364, 2: 158}

{1: 364, 2: 159}

{1: 364, 2: 160}

{1: 365, 2: 160}

{1: 365, 2: 161}

{1: 366, 2: 161}

{1: 367, 2: 161}

{1: 368, 2: 161}

{1: 369, 2: 161}

{1: 370, 2: 161}

{1: 371, 2: 161}

{1: 372, 2: 161}

{1: 372, 2: 162}

{1: 372, 2: 163}

{1: 372, 2: 164}

{1: 373, 2: 164}

{1: 374, 2: 164}

{1: 375, 2: 164}

{1: 375, 2

{1: 668, 2: 273}

{1: 669, 2: 273}

{1: 670, 2: 273}

{1: 671, 2: 273}

{1: 672, 2: 273}

{1: 672, 2: 274}

{1: 673, 2: 274}

{1: 674, 2: 274}

{1: 675, 2: 274}

{1: 676, 2: 274}

{1: 677, 2: 274}

{1: 678, 2: 274}

{1: 678, 2: 275}

{1: 678, 2: 276}

{1: 679, 2: 276}

{1: 679, 2: 277}

{1: 680, 2: 277}

{1: 680, 2: 278}

{1: 681, 2: 278}

{1: 682, 2: 278}

{1: 683, 2: 278}

{1: 684, 2: 278}

{1: 684, 2: 279}

{1: 685, 2: 279}

{1: 686, 2: 279}

{1: 686, 2: 280}

{1: 687, 2: 280}

{1: 688, 2: 280}

{1: 689, 2: 280}

{1: 689, 2: 281}

{1: 690, 2: 281}

{1: 691, 2: 281}

{1: 692, 2: 281}

{1: 693, 2: 281}

{1: 693, 2: 282}

{1: 693, 2: 283}

{1: 694, 2: 283}

{1: 695, 2: 283}

{1: 695, 2: 284}

{1: 696, 2: 284}

{1: 696, 2: 285}

{1: 697, 2: 285}

{1: 698, 2: 285}

{1: 699, 2: 285}

{1: 700, 2: 285}

{1: 701, 2: 285}

{1: 702, 2: 285}

{1: 703, 2: 285}

{1: 704, 2: 285}

{1: 705, 2: 285}

{1: 705, 2: 286}

{1: 706, 2: 286}

{1: 707, 2: 286}

{1: 707, 2: 287}

{1: 708, 2: 287}

{1: 709, 2

In [ ]:
def softmax(w):
    """Calculate the softmax of a list of numbers w.

    Parameters
    ----------
    w : list of numbers

    Return
    ------
    a list of the same length as w of non-negative numbers

    Examples
    --------
    >>> softmax([0.1, 0.2])
    array([ 0.47502081,  0.52497919])
    >>> softmax([-0.1, 0.2])
    array([ 0.42555748,  0.57444252])
    >>> softmax([0.9, -10])
    array([  9.99981542e-01,   1.84578933e-05])
    >>> softmax([0, 10])
    array([  4.53978687e-05,   9.99954602e-01])
    """
    e = np.exp(np.array(w))
    dist = e / np.sum(e)
    return dist

In [ ]:
softmax([0])

In [ ]:
softmax([0.01,0.8,0.19])

In [ ]:
softmax([0.01,0.01,0.19])

In [ ]:
softmax([0.01,0,0.19])

In [ ]:
softmax([0.01,-0.8,0.19])